# Linear regression
---


## Nathan Karst

<h1>Overview</h1>

<ul>
<li>Linear regression is a bread-and-butter technique from classical statistics, and so it makes a natural starting point for investigating a fundamental question: 
</li>

<br/>
<br/>

<font size="6"><center>How is a big data mindset different <br> than a traditional statistical mindset?</center></font>


# Review

* In linear regression, we have a variable $Y$ that we're trying to predict using the information from a collection of variables $X_1, X_2, \ldots, X_p$. 


* In traditional statistics, we often call $Y$ the *dependent variable* and the $X_i$ the *indepdent variables*.


* In big data, we often call $Y$ the *target* and the $X_i$ the *predictors* or *inputs*.


* We assume that a very particular relationship holds: 
<br>
<br>
$$Y = \beta_0 + \beta_1 X_1 + \beta_2 X_2 + \ldots + \beta_p X_p + \epsilon.$$
<br>
* All of the information in the model is contained in the $\beta$s, called the *coefficients* of the linear model. 


* For every increase in $X_i$ by 1 unit, $Y$ on average increases by $\beta_i$. 


* The variable $\epsilon$ contains all the information in $Y$ that is not captured by the linear trends. We hope that this is small -- sometimes it is, and sometimes it isn't.

# Review

* If we make a prediction $\hat{Y}_i$ for a given observation $Y_i$, then the error is just the difference between the two: 
<br>
<br>
$$\mbox{error}_i = Y_i - \hat{Y}_i.$$
<br>
* Under this definition, error is positive if we *underestimate* and negative if we *overestimate*.


* Because errors can be positive and negative, we can't just sum them to get a measure of total error over all predictions -- positive and negative errors could cancel out to 0, even if the model were not perfect! 


* We instead look at the **sum of squared errors** (SSE): 
<br>
<br>
$$ \mbox{SSE} = \sum_{i=1}^n (Y_i - \hat{Y}_i)^2.$$
<br>
* Our linear regression model produces estimates $\hat{\beta}_0, \hat{\beta}_1, \ldots, \hat{\beta}_p$ that minimize SSE. 

# Old and new

* But isn't all of this just what we learned in traditional statistics? So how is big data any different? 


* In classical statistics, 
    * We have very limited data.
    * We want to create a rule to *explain* how $Y$ depends on the $X_i$. 
    * We assess model performance based on how well our model fits *old* data.
    
    
* In big data,
    * We typically have *lots* of data.
    * We want to create a rule to *predict* how $Y$ will be have.
    * We assess model performance based on how accurately it predicts *new* data.

# The data analytics tool flow

![alt text](https://www.dropbox.com/s/wlc6ffqb8k96h7b/6_training_test_flow.png?raw=1)

In [8]:
import pandas as pd
import seaborn as sns
import BabsonAnalytics
import numpy as np
%matplotlib inline

# Load and manage

In [15]:
df = pd.read_csv('../data/BostonHousing.csv')
df.CHAS = df.CHAS.astype("category")
df = df.drop('CAT. MEDV',axis=1)
df = BabsonAnalytics.makeDummies(df)
df.head()

,CRIM,ZN,INDUS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,LSTAT,MEDV,CHAS_1
0,0.00632,18.0,2.31,0.538,6.575,65.2,4.0900,1,296,15.3,4.98,24.0,0.0
1,0.02731,0.0,7.07,0.469,6.421,78.9,4.9671,2,242,17.8,9.14,21.6,0.0
2,0.02729,0.0,7.07,0.469,7.185,61.1,4.9671,2,242,17.8,4.03,34.7,0.0
3,0.03237,0.0,2.18,0.458,6.998,45.8,6.0622,3,222,18.7,2.94,33.4,0.0
4,0.06905,0.0,2.18,0.458,7.147,54.2,6.0622,3,222,18.7,5.33,36.2,0.0


# Partition

* We need to split the data set into two pieces: a training set that we will use to build the model, and a test set we'll use to evaluate model performance.
* Typical splits range anywhere from 60-40 to 80-20, with the larger piece going to training.


In [10]:
train = df.sample(frac=0.8) # sample 80% of the rows from df
test = df.drop(train.index) # drop all that are now in training

trainTarget = train.pop('MEDV') # pop out the target from df...
testTarget = test.pop('MEDV') # ...we'll need these separately later

* Notice that at this point, the target is no longer found in `train` or `test`. 

# Build

* Now that we have the training and test data set up, we're ready to build the model.
* This phase looks different for each model type (e.g., linear regression).
* The parts are always the same:
    * Create an empty model.
    * Fit the model to the data.
    * (Inspect the model if you want to...)

In [11]:
from sklearn import linear_model 
model = linear_model.LinearRegression(fit_intercept=True) # build an empty linear model
model.fit(train,trainTarget) # fit the model to the training data
BabsonAnalytics.inspectLinearModel(train,trainTarget,model) # inspect the model

             Estimate  p-value
Predictor                     
(Intercept)     39.46    0.000
CRIM            -0.12    0.003
ZN               0.04    0.003
INDUS           -0.04    0.585
NOX            -15.26    0.001
RM               3.73    0.000
AGE             -0.01    0.501
DIS             -1.51    0.000
RAD              0.24    0.001
TAX             -0.01    0.003
PTRATIO         -0.90    0.000
LSTAT           -0.49    0.000
CHAS_1           2.97    0.002





# Predict

* With the model in hand, we can now make some predictions. 


* Remember, we're always predicting on the test data frame.



In [12]:
predictions = model.predict(test)
predictions[:5]

array([ 30.29295506,  25.27340275,  22.7539548 ,  19.34059394,  17.69168114])

# Evaluate

* We now have to figure out whether our predictions are any good.


* The definition of "good" is different for numerical and categorical variables.


* For numeric variables, we have two measures of quality, MAPE and RMSE:
<br>
<br>
$$\mbox{MAPE} = \sum_{i=1}^N \frac{|Y_i - \hat{Y}_i|}{Y_i}, \quad \quad \mbox{RMSE} = \sqrt{\sum_{i=1}^N (Y_i - \hat{Y}_i)^2}$$
<br>
* MAPE is measured in terms of percent deviation, and RMSE is measured in whatever the units of the target are, e.g., here thousands of dollars.

In [13]:
error = testTarget - predictions
rmse = np.sqrt(np.mean(error**2))
mape = np.mean(np.abs(error/testTarget))
print('RMSE: ', rmse)
print('MAPE: ', mape)

RMSE:  5.07009928237
MAPE:  0.14863863423730617


# Your turn

* Create a new notebook.


* Load the data located in ToyotaCorolla.csv.


* Change `Automatic` and `Met_Color` to categorical variables (if they are not already).


* Creat dummy variables.


* Create a linear model for `Price`, and fit this model to the training data.


* Using this model, make predictions for the observations in the test partition. 


* Compute the MAPE and RMSE associated with your predictions.


* Now go back and generate your dummy variables again, but using `forRegression=False` instead of `forRegression=True`. What happens to your model?

# Variable Selection

* Given a collection of predctions, how can we find the *best* model we could construct? 


* Imagine we have a data set with 50 columns. 


* There are $2^{50} \approx 1 \times 10^{15}$ linear models we could make with these variables. 


* This is a large-ish number. If we could evaluate one model per second, it would take us 31 million years to get through them all.


* If we have 60 variables (just 10 more!), the same process would take 36 billion years.


* Obviously, checking all possible models is not feasible. We need to be more clever.



# Variable Selection

* Here we'll use *recursive feature elimination* (RFE) to prune back the number of variables we're using. 


* We first look at all variables, find the one that contributes least to the predictive power of the model, and remove it.

* We then look at the model created by all the remaining variables, find the one that contributes the least, and remove it. 


* We continue doing this until we have a model with $k$ variables, where $k$ is a number we have specified beforehand. 


* The hope here is that we'll end up with a smaller collection of variables that do about as good of a job at predicting the target as the full data set. 


* A smaller number of predictors are cheaper and easier to collect, clean, and store, and fewer data points means faster fitting and predicting. 


* Moreover, we can clearly draw a line between the variables that matter and the variables that don't. 

In [14]:
from sklearn import feature_selection
model = linear_model.LinearRegression();
rfe = feature_selection.RFE(model,4)
rfe.fit(train,trainTarget)

BabsonAnalytics.inspectLinearModel(train=train,trainTarget=trainTarget,model=rfe)

predictions = rfe.predict(test)

error = testTarget - predictions
rmse = np.sqrt(np.mean(error**2))
mape = np.mean(np.abs(error/testTarget))
print('RMSE: ', rmse)
print('MAPE: ', mape)

             Estimate  p-value
Predictor                     
(Intercept)     12.28    0.009
NOX            -18.49    0.000
RM               6.63    0.000
PTRATIO         -1.17    0.000
CHAS_1           4.12    0.000



RMSE:  6.20447508867
MAPE:  0.16713999750546887


* Notice that now all variables are significant, and that the MAPE and RMSE aren't much different from those of the full model.

# Your turn 

* Perform RFE on your linear model for the price of a Toyota Corolla.


* Which variables are kept? Which are thrown away?


* Does the performance of your model on test degrade when you use 6 predictors? 4 predictors? 2 predictors? 

